# Primera versió del projecte. Anàl.lisi d'un discurs, extracció de les paraules clau i representació gràfica dels resultats.

Prenent com a model aquest <a href="http://www.nytimes.com/interactive/2012/09/06/us/politics/convention-word-counts.html?_r=0">artícle</a> on s'analitzen els discursos de demòcrates i republicans a les eleccions dels EUA de 2012 intentem fer un primer projecte que serveix com a exercici per establir les passes per fer un projecte similar en el context polític espanyol.

## Lectura d'un fitxer pdf

Mètode per extreure el texte d'un fitxer en format pdf.

(És necessari instal.lar PDFMiner ( http://www.unixuser.org/~euske/python/pdfminer/ ). Es pot instal.lar desde la linea de comandes executant: pip install pdfminer )

In [1]:
# http://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    
    # http://stackoverflow.com/questions/25315566/unicodedecodeerror-in-nltks-word-tokenize-despite-i-forced-the-encoding
    text = text.decode('utf-8').lower()
    return text

#### Extracció del texte del discurs: "Discurso de Mariano Rajoy en la clausura de la Convención Nacional, (Madrid, 25/01/2015)"

 (descarregat de la web de PP: http://www.pp.es/sites/default/files/documentos/15.01.25_discurso_rajoy_-_convencion_nacional_definitivo.pdf ).

In [2]:
text = convert_pdf_to_txt("../data/pp/15.01.25_discurso_rajoy_-_convencion_nacional_definitivo.pdf")
print text

IOError: [Errno 2] No such file or directory: '../data/pp/15.01.25_discurso_rajoy_-_convencion_nacional_definitivo.pdf'

## Anàlisi del text del discurs

### Mètodes per netejar el texte: eliminar paraules i signes que no són rellevants.

In [3]:
import re
import string
from nltk.tokenize import word_tokenize

# Mètode per eliminar signes de puntuació
def tokenized_doc_no_punctuation(tokenized_doc):
    
    regex = re.compile('[%s]' % re.escape(string.punctuation)) 

    tokenized_words_no_punctuation = []

    for token in tokenized_doc: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            tokenized_words_no_punctuation.append(new_token)

    return tokenized_words_no_punctuation

# Mètode per eliminar xifres numèriques
def tokenized_doc_no_numbers(tokenized_doc):
    
    regex = re.compile(re.compile(r'[()|0-9]')) 

    tokenized_words_no_numbers = []

    for token in tokenized_doc: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            tokenized_words_no_numbers.append(new_token)

    return tokenized_words_no_numbers

In [4]:
# Conjunt de paraules (tokens) del text
tokenized_words = word_tokenize(text)
print tokenized_words

[u'discurso', u'de', u'mariano', u'rajoy', u'convenci\xf3n', u'nacional', u'del', u'pp', u'\u201cjuntos', u'por', u'un', u'gran', u'pa\xeds\u201d', u'madrid', u',', u'25', u'de', u'enero', u'de', u'2015', u'g\xe9nova', u',', u'13', u'-', u'28004', u'madrid', u'.', u'telfs', u':', u'(', u'91', u')', u'557', u'73', u'58', u'/', u'59.', u'fax', u':', u'(', u'91', u')', u'319', u'02', u'81.', u's\xf3lo', u'ser\xe1n', u'v\xe1lidas', u'las', u'palabras', u'pronunciadas', u'por', u'el', u'orador', u'1', u'queridas', u'amigas', u'y', u'amigos', u':', u'quiero', u'daros', u'las', u'gracias', u'una', u'vez', u'm\xe1s', u'y', u'felicitaros', u'por', u'esta', u'convenci\xf3n', u'.', u'ha', u'sido', u'una', u'gran', u'convenci\xf3n', u'.', u'quiero', u'felicitar', u'expresamente', u'a', u'mar\xeda', u'dolores', u',', u'a', u'carlos', u',', u'a', u'javier', u'y', u'a', u'esteban', u'.', u'a', u'todas', u'las', u'personas', u'que', u'han', u'ayudado', u'al', u'\xe9xito', u'de', u'esta', u'convocatori

In [5]:
# Eliminar els signes de puntuació i les xifres numèriques

tokenized_words_no_punctuation = tokenized_doc_no_punctuation(tokenized_words)
print tokenized_words_no_punctuation

tokenized_words_no_numbers = tokenized_doc_no_numbers(tokenized_words_no_punctuation)
print tokenized_words_no_numbers


[u'discurso', u'de', u'mariano', u'rajoy', u'convenci\xf3n', u'nacional', u'del', u'pp', u'\u201cjuntos', u'por', u'un', u'gran', u'pa\xeds\u201d', u'madrid', u'25', u'de', u'enero', u'de', u'2015', u'g\xe9nova', u'13', u'28004', u'madrid', u'telfs', u'91', u'557', u'73', u'58', u'59', u'fax', u'91', u'319', u'02', u'81', u's\xf3lo', u'ser\xe1n', u'v\xe1lidas', u'las', u'palabras', u'pronunciadas', u'por', u'el', u'orador', u'1', u'queridas', u'amigas', u'y', u'amigos', u'quiero', u'daros', u'las', u'gracias', u'una', u'vez', u'm\xe1s', u'y', u'felicitaros', u'por', u'esta', u'convenci\xf3n', u'ha', u'sido', u'una', u'gran', u'convenci\xf3n', u'quiero', u'felicitar', u'expresamente', u'a', u'mar\xeda', u'dolores', u'a', u'carlos', u'a', u'javier', u'y', u'a', u'esteban', u'a', u'todas', u'las', u'personas', u'que', u'han', u'ayudado', u'al', u'\xe9xito', u'de', u'esta', u'convocatoria', u'y', u'a', u'todos', u'los', u'participantes', u'en', u'los', u'distintos', u'debates', u'que', u'h

### Eliminació de paraules "comunes"

- Stopwords en espanyol

- El document utilizat, "15.01.25_discurso_rajoy_-_convencion_nacional_definitivo.pdf", conté un peu de pàgina (a cada pàgina) amb el següent texte:
        "génova, 13 - 28004 madrid. telfs: (91) 557 73 58 / 59. fax: (91) 319 02 81.  

        sólo serán válidas las palabras pronunciadas por el orador "
        
  TODO: Com detectar paraules o paràgrafs com aquest en els textes i eliminar aquestes repeticions?

In [6]:
from nltk.corpus import stopwords

# Mètode per eliminar les paraules que pertanyen a un conjunt de paraules donat (custom_set_of_words) 
def remove_custom_set_of_words(tokenized_doc, custom_set_of_words):
    return [ word for word in tokenized_doc if word not in custom_set_of_words ]

#### Eliminació de stopwords en espanyol

In [7]:
# Conjunt de paraules stopwords de l'espanyol del paquet nltk
print stopwords.words('spanish')

[u'de', u'la', u'que', u'el', u'en', u'y', u'a', u'los', u'del', u'se', u'las', u'por', u'un', u'para', u'con', u'no', u'una', u'su', u'al', u'lo', u'como', u'm\xe1s', u'pero', u'sus', u'le', u'ya', u'o', u'este', u's\xed', u'porque', u'esta', u'entre', u'cuando', u'muy', u'sin', u'sobre', u'tambi\xe9n', u'me', u'hasta', u'hay', u'donde', u'quien', u'desde', u'todo', u'nos', u'durante', u'todos', u'uno', u'les', u'ni', u'contra', u'otros', u'ese', u'eso', u'ante', u'ellos', u'e', u'esto', u'm\xed', u'antes', u'algunos', u'qu\xe9', u'unos', u'yo', u'otro', u'otras', u'otra', u'\xe9l', u'tanto', u'esa', u'estos', u'mucho', u'quienes', u'nada', u'muchos', u'cual', u'poco', u'ella', u'estar', u'estas', u'algunas', u'algo', u'nosotros', u'mi', u'mis', u't\xfa', u'te', u'ti', u'tu', u'tus', u'ellas', u'nosotras', u'vosostros', u'vosostras', u'os', u'm\xedo', u'm\xeda', u'm\xedos', u'm\xedas', u'tuyo', u'tuya', u'tuyos', u'tuyas', u'suyo', u'suya', u'suyos', u'suyas', u'nuestro', u'nuestra', 

In [8]:
tokenized_words_no_stopwords = remove_custom_set_of_words(tokenized_words_no_numbers, stopwords.words('spanish'))
print tokenized_words_no_stopwords

[u'discurso', u'mariano', u'rajoy', u'convenci\xf3n', u'nacional', u'pp', u'\u201cjuntos', u'gran', u'pa\xeds\u201d', u'madrid', u'enero', u'g\xe9nova', u'madrid', u'telfs', u'fax', u's\xf3lo', u'v\xe1lidas', u'palabras', u'pronunciadas', u'orador', u'queridas', u'amigas', u'amigos', u'quiero', u'daros', u'gracias', u'vez', u'felicitaros', u'convenci\xf3n', u'sido', u'gran', u'convenci\xf3n', u'quiero', u'felicitar', u'expresamente', u'mar\xeda', u'dolores', u'carlos', u'javier', u'esteban', u'todas', u'personas', u'ayudado', u'\xe9xito', u'convocatoria', u'participantes', u'distintos', u'debates', u'd\xedas', u'vosotros', u'sido', u'gran', u'trabajo', u'pp', u'primer', u'partido', u'espa\xf1a', u'militantes', u'votos', u'instituciones', u'gobierna', u'responsabilidad', u'espaldas', u'tanta', u'confianza', u'encomendada', u'recibido', u'espa\xf1oles', u'responsabilidad', u'mayor', u'nunca', u'asustado', u'responsabilidad', u'nunca', u'mirado', u'hacia', u'lado', u'escurrido', u'bulto',

#### Eliminació d'altres paraules no significatives

Aquesta part és un exemple simple (handmade) d'eliminació d'algunes paraules no significatives.
Veient els resultats posteriors em sorgeix la necessitat (i el dubte) de trobar un procediment o eines com la librería nltk per eliminar paraules (i potser expressions) com: "si" (conjunció), "toda", "sido", ...

In [9]:
# Eliminar les paraules del peu de pàgina del document "15.01.25_discurso_rajoy_-_convencion_nacional_definitivo.pdf"
# El peu de pàgina conté aquest text:
#  "génova, 13 - 28004 madrid. telfs: (91) 557 73 58 / 59. fax: (91) 319 02 81.
#   sólo serán válidas las palabras pronunciadas por el orador"
# Conjunt de paraules d'aquest text:
# custom_set_of_words = ['génova', 'madrid', 'telfs', 'fax', 'sólo', 'válidas', 'palabras', 'pronunciadas', 'orador']
custom_set_of_words = [u'g\xe9nova', 'madrid', 'telfs', 'fax', u's\xf3lo', u'v\xe1lidas', 'palabras', 'pronunciadas', 'orador']

tokenized_words_final = remove_custom_set_of_words(tokenized_words_no_stopwords, custom_set_of_words)
print tokenized_words_final

[u'discurso', u'mariano', u'rajoy', u'convenci\xf3n', u'nacional', u'pp', u'\u201cjuntos', u'gran', u'pa\xeds\u201d', u'enero', u'queridas', u'amigas', u'amigos', u'quiero', u'daros', u'gracias', u'vez', u'felicitaros', u'convenci\xf3n', u'sido', u'gran', u'convenci\xf3n', u'quiero', u'felicitar', u'expresamente', u'mar\xeda', u'dolores', u'carlos', u'javier', u'esteban', u'todas', u'personas', u'ayudado', u'\xe9xito', u'convocatoria', u'participantes', u'distintos', u'debates', u'd\xedas', u'vosotros', u'sido', u'gran', u'trabajo', u'pp', u'primer', u'partido', u'espa\xf1a', u'militantes', u'votos', u'instituciones', u'gobierna', u'responsabilidad', u'espaldas', u'tanta', u'confianza', u'encomendada', u'recibido', u'espa\xf1oles', u'responsabilidad', u'mayor', u'nunca', u'asustado', u'responsabilidad', u'nunca', u'mirado', u'hacia', u'lado', u'escurrido', u'bulto', u'mirado', u'realidad', u'cara', u'cara', u'puesto', u'manos', u'obra', u'transformarla', u'vivido', u'etapa', u'gran', u

### Stemming

(Stemming es un método para reducir una palabra a su raíz o (en inglés) a un stem o lema [ http://es.wikipedia.org/wiki/Stemming ] )

In [10]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter = PorterStemmer()
snowball = SnowballStemmer('spanish')
wordnet = WordNetLemmatizer()

tokenized_words_stemmed = []

for word in tokenized_words_final:
   # tokenized_words_stemmed.append(porter.stem(word))
   tokenized_words_stemmed.append(snowball.stem(word)) # requires 'corpora/wordnet' -> nltk.download()
   # tokenized_words_stemmed.append(wordnet.lemmatize(word)) # requires 'corpora/wordnet' -> nltk.download()

print tokenized_words_stemmed

[u'discurs', u'marian', u'rajoy', u'convencion', u'nacional', u'pp', u'\u201cjunt', u'gran', u'pais\u201d', u'ener', u'quer', u'amig', u'amig', u'quier', u'dar', u'graci', u'vez', u'felicitar', u'convencion', u'sid', u'gran', u'convencion', u'quier', u'felicit', u'expres', u'mar', u'dolor', u'carl', u'javi', u'esteb', u'tod', u'person', u'ayud', u'exit', u'convocatori', u'particip', u'distint', u'debat', u'dias', u'vosotr', u'sid', u'gran', u'trabaj', u'pp', u'prim', u'part', u'espa\xf1', u'milit', u'vot', u'institu', u'gobiern', u'respons', u'espald', u'tant', u'confianz', u'encomend', u'recib', u'espa\xf1ol', u'respons', u'mayor', u'nunc', u'asust', u'respons', u'nunc', u'mir', u'haci', u'lad', u'escurr', u'bult', u'mir', u'realid', u'car', u'car', u'puest', u'man', u'obra', u'transform', u'viv', u'etap', u'gran', u'dificult', u'vosotr', u'amig', u'amig', u'part', u'popul', u'respond', u'siempr', u'enterez', u'determin', u'respons', u'voy', u'dec', u'patriot', u'hac', u'falt', u'patr

In [11]:
# Mètode per omplir el diccionari de stems
stem_dict = {}
for word in tokenized_words_final:
    stem_word = snowball.stem(word)
    if stem_word in stem_dict:
        if word in stem_dict[stem_word]["words"]:
            stem_dict[stem_word]["words"][word] += 1
        else:
            stem_dict[stem_word]["words"][word] = 1
        stem_dict[stem_word]["freq"] += 1
    else:
        stem_dict[stem_word] = {'words':{word:1},'freq':1,'rep':word}
    
    rep = stem_dict[stem_word]["rep"]
    if ( stem_dict[stem_word]["words"][word] > stem_dict[stem_word]["words"][rep] ):
        stem_dict[stem_word]["rep"] = word

print stem_dict

{u'\u2013y': {'freq': 2, 'rep': u'\u2013y', 'words': {u'\u2013y': 2}}, u'espald': {'freq': 1, 'rep': u'espaldas', 'words': {u'espaldas': 1}}, u'\xbfy': {'freq': 4, 'rep': u'\xbfy', 'words': {u'\xbfy': 4}}, u'segu': {'freq': 14, 'rep': u'seguir', 'words': {u'seguimos': 1, u'seguir\xe1': 3, u'seguir': 7, u'seguiremos': 2, u'seguir\xe9': 1}}, u'consum': {'freq': 1, 'rep': u'consumir\xe1', 'words': {u'consumir\xe1': 1}}, u'encontr': {'freq': 1, 'rep': u'encontrar', 'words': {u'encontrar': 1}}, u'\xbfo': {'freq': 1, 'rep': u'\xbfo', 'words': {u'\xbfo': 1}}, u'ayud': {'freq': 3, 'rep': u'ayudado', 'words': {u'ayudado': 1, u'ayudar': 1, u'ayudas': 1}}, u'ciudad': {'freq': 1, 'rep': u'ciudades', 'words': {u'ciudades': 1}}, u'concept': {'freq': 1, 'rep': u'concepto', 'words': {u'concepto': 1}}, u'confund': {'freq': 1, 'rep': u'confunden', 'words': {u'confunden': 1}}, u'correg': {'freq': 1, 'rep': u'corregirlas', 'words': {u'corregirlas': 1}}, u'encaden': {'freq': 1, 'rep': u'encadenamos', 'word

## Freqüències de les paraules més significatives

In [12]:
from collections import Counter

# Mètode que retorna un Counter amb la freqüència de les paraules d'un array de paraules
def word_freq( tokenized_doc ):
    
    tf = Counter()
    for word in tokenized_doc:
        tf[word] +=1
    return tf

import operator
# Mètode que retorna el diccionari ordenat per la freqüència de les paraules
def word_freq_dict( dict_doc ):
    # http://stackoverflow.com/questions/613183/sort-a-python-dictionary-by-value
    return sorted(dict_doc.items(), key=operator.itemgetter(1), reverse=True)

In [13]:
wordCounter = word_freq(tokenized_words_final)
# print wordCounter

stem_dict_sorted = word_freq_dict( stem_dict )
print stem_dict_sorted

[(u'espa\xf1', {'freq': 56, 'rep': u'espa\xf1a', 'words': {u'espa\xf1a': 56}}), (u'espa\xf1ol', {'freq': 39, 'rep': u'espa\xf1oles', 'words': {u'espa\xf1ola': 2, u'espa\xf1ol': 1, u'espa\xf1oles': 36}}), (u'part', {'freq': 36, 'rep': u'partido', 'words': {u'partido': 30, u'parte': 4, u'partes': 2}}), (u'hac', {'freq': 32, 'rep': u'hacer', 'words': {u'hacer': 13, u'hacerlo': 4, u'haciendo': 2, u'haci\xe9ndolo': 2, u'hacemos': 2, u'hace': 7, u'hacen': 1, u'hacerles': 1}}), (u'pais', {'freq': 28, 'rep': u'pa\xeds', 'words': {u'pa\xeds': 24, u'pa\xedses': 4}}), (u'ahor', {'freq': 25, 'rep': u'ahora', 'words': {u'ahora': 25}}), (u'pod', {'freq': 24, 'rep': u'podemos', 'words': {u'pod\xe9is': 2, u'pod\xedamos': 1, u'podemos': 18, u'poder': 2, u'podamos': 1}}), (u'dec', {'freq': 24, 'rep': u'decir', 'words': {u'dec\xedan': 1, u'decirnos': 1, u'dec\xeda': 1, u'decirles': 1, u'decir': 14, u'decirlo': 6}}), (u'mejor', {'freq': 23, 'rep': u'mejor', 'words': {u'mejores': 3, u'mejoran': 1, u'mejor'

#### Seleccionem les 20 paraules amb més ocurrències en el discurs

In [14]:
diccionari_final = wordCounter.most_common(20)
print diccionari_final

diccionari_stemmed_final = stem_dict_sorted[:20]
print diccionari_stemmed_final

[(u'espa\xf1a', 56), (u'espa\xf1oles', 36), (u'partido', 30), (u'ahora', 25), (u'pa\xeds', 24), (u'podemos', 18), (u'sido', 17), (u'bien', 17), (u'cambio', 17), (u'a\xf1o', 15), (u'decir', 14), (u'popular', 14), (u'mejor', 14), (u'hacer', 13), (u'a\xf1os', 13), (u'si', 13), (u'cosas', 13), (u'elecciones', 12), (u'toda', 12), (u'empleo', 12)]
[(u'espa\xf1', {'freq': 56, 'rep': u'espa\xf1a', 'words': {u'espa\xf1a': 56}}), (u'espa\xf1ol', {'freq': 39, 'rep': u'espa\xf1oles', 'words': {u'espa\xf1ola': 2, u'espa\xf1ol': 1, u'espa\xf1oles': 36}}), (u'part', {'freq': 36, 'rep': u'partido', 'words': {u'partido': 30, u'parte': 4, u'partes': 2}}), (u'hac', {'freq': 32, 'rep': u'hacer', 'words': {u'hacer': 13, u'hacerlo': 4, u'haciendo': 2, u'haci\xe9ndolo': 2, u'hacemos': 2, u'hace': 7, u'hacen': 1, u'hacerles': 1}}), (u'pais', {'freq': 28, 'rep': u'pa\xeds', 'words': {u'pa\xeds': 24, u'pa\xedses': 4}}), (u'ahor', {'freq': 25, 'rep': u'ahora', 'words': {u'ahora': 25}}), (u'pod', {'freq': 24, 're

In [15]:
print "Llistat de les paraules amb més ocurrències al discurs:\n"
for word in diccionari_final:
    print word[0] +"\t", ": %d" % word[1]
    
print "\nLlistat de les paraules (arrels) amb més ocurrències al discurs:\n"
for word_stemmed in diccionari_stemmed_final:
    print word_stemmed[0] +"\t", ": %d" % word_stemmed[1]['freq'], "["+ word_stemmed[1]['rep']+"]", word_stemmed[1]['words']

Llistat de les paraules amb més ocurrències al discurs:

españa	: 56
españoles	: 36
partido	: 30
ahora	: 25
país	: 24
podemos	: 18
sido	: 17
bien	: 17
cambio	: 17
año	: 15
decir	: 14
popular	: 14
mejor	: 14
hacer	: 13
años	: 13
si	: 13
cosas	: 13
elecciones	: 12
toda	: 12
empleo	: 12

Llistat de les paraules (arrels) amb més ocurrències al discurs:

españ	: 56 [españa] {u'espa\xf1a': 56}
español	: 39 [españoles] {u'espa\xf1ola': 2, u'espa\xf1ol': 1, u'espa\xf1oles': 36}
part	: 36 [partido] {u'partido': 30, u'parte': 4, u'partes': 2}
hac	: 32 [hacer] {u'hacer': 13, u'hacerlo': 4, u'haciendo': 2, u'haci\xe9ndolo': 2, u'hacemos': 2, u'hace': 7, u'hacen': 1, u'hacerles': 1}
pais	: 28 [país] {u'pa\xeds': 24, u'pa\xedses': 4}
ahor	: 25 [ahora] {u'ahora': 25}
pod	: 24 [podemos] {u'pod\xe9is': 2, u'pod\xedamos': 1, u'podemos': 18, u'poder': 2, u'podamos': 1}
dec	: 24 [decir] {u'dec\xedan': 1, u'decirnos': 1, u'dec\xeda': 1, u'decirles': 1, u'decir': 14, u'decirlo': 6}
mejor	: 23 [mejor] {u'mej

### Exportar un fitxer json amb els resultats obtinguts 

In [16]:
import json
wordCounterJson = json.dumps(diccionari_final)
print wordCounterJson

with open('diccionari_final.json', 'w') as outfile:
    json.dump(diccionari_final, outfile)

[["espa\u00f1a", 56], ["espa\u00f1oles", 36], ["partido", 30], ["ahora", 25], ["pa\u00eds", 24], ["podemos", 18], ["sido", 17], ["bien", 17], ["cambio", 17], ["a\u00f1o", 15], ["decir", 14], ["popular", 14], ["mejor", 14], ["hacer", 13], ["a\u00f1os", 13], ["si", 13], ["cosas", 13], ["elecciones", 12], ["toda", 12], ["empleo", 12]]


## Representació dels resultats amb D3

- Bar chart que carrega un fitxer json (a partir d'un exemple de la classe de visualització)

In [17]:
%%html
<style>
.chart div {
  font: 10px sans-serif;
  background-color: steelblue;
  text-align: right;
  padding: 3px;
  margin: 1px;
  color: white;
}
</style>

<div id="my_chart" class="chart"></div>

In [18]:
%%javascript
    // We load the d3.js library from the Web.
    require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
    require(["d3"], function(d3) {
    
    var div = d3.select("#my_chart");
        
    d3.json("diccionari_final.json", function(data) {
        
        div.selectAll("div")
            .data(data)
            .enter().append("div")
            .style("width", function(d) { return d[1]*10 + "px"; })
            .text(function(d) { return d[0] + " (" + d[1] + ")"; });
    });    
});

- Un altre exemple de Bar Chart ( a partir de https://gist.github.com/tristanwietsma/6047126 )

    [ En construcció     :-)     ]

In [19]:
%%html
<style>
    text {
        font: sans-serif;
    }

    .bar { fill: steelblue; }
</style>

<div id="my_chart2"></div>

In [20]:
%%javascript
    // We load the d3.js library from the Web.
    require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
    require(["d3"], function(d3) {
 
        var w = 600,
        h = 400;        
        
        var svg = d3.select("#my_chart2")
            .append("svg")
            .attr("width", w)
            .attr("height", h);
    
        d3.json("diccionari_final.json", function(json) {
    
            var data = json;

            var max_n = 0;
            for (var i = 0; i < data.length; i++) {
                max_n = Math.max(data[i][1], max_n);
            }
        
            var dx = w / max_n;
            var dy = h / data.length;
            var sp = 5;
    
            // bars
            var bars = svg.selectAll(".bar")
                .data(data)
                .enter()
                .append("rect")
                .attr("class", function(d, i) {return "bar " + d[0];})
                .attr("x", function(d, i) {return 0;})
                .attr("y", function(d, i) {return (dy+sp)*i;})
                .attr("width", function(d, i) {return dx*d[1];})
                .attr("height", dy);
    
            // labels
            var text = svg.selectAll("text")
                .data(data)
                .enter()
                .append("text")
                .attr("class", function(d, i) {return "label " + d[0];})
                .attr("x", function(d, i) {return sp;})
                .attr("y", function(d, i) {return (dy+sp)*i+(2/3)*dy;})
                .attr("font-size", dy + "px")
                .style("font-weight", "bold")
                .text( function(d) {return d[0] + " (" + d[1]  + ")";});
        });
});

- Exemple de Bubble Chart ( extret amb modificacions de: http://bl.ocks.org/mbostock/4063269 )

In [21]:
%%html
<style>
    text {
        font: sans-serif;
    }
</style>

<div id="my_first_bubble_chart"></div>

In [22]:
%%javascript
    // We load the d3.js library from the Web.
    require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
    require(["d3"], function(d3) {
 
    var diameter = 960,
        format = d3.format(",d"),
        // https://github.com/mbostock/d3/wiki/Ordinal-Scales#categorical-colors
        color = d3.scale.category20c();

    var bubble = d3.layout.pack()
        .sort(null)
        .size([diameter, diameter])
        .padding(1.5);

    var svg = d3.select("#my_first_bubble_chart").append("svg")
        .attr("width", diameter)
        .attr("height", diameter)
        .attr("class", "bubble");

    d3.json("diccionari_final.json", function(data) {
      var node = svg.selectAll(".node")
          .data(bubble.nodes(processData(data))
          .filter(function(d) { return !d.children; }))
        .enter().append("g")
          .attr("class", "node")
          .attr("transform", function(d) { return "translate(" + d.x + "," + d.y + ")"; });

      node.append("title")
          .text(function(d) { return d.className + ": " + format(d.value); });

      node.append("circle")
          .attr("r", function(d) { return d.r; })
          .style("fill", function(d) { return color(d.packageName); });

      node.append("text")
          .attr("dy", ".3em")
          .style("text-anchor", "middle")
          .style("font-size", function(d) { return Math.round(d.r/3)+"px"; })
          .text(function(d) { return d.className; });
    });

    function processData(data) { 

        var newDataSet = [];
        for (var i = 0; i < data.length; i++) { 
            newDataSet.push({packageName: "PP Top Words", className: data[i][0], value: data[i][1]});
        }
        return {children: newDataSet};
    }

    d3.select(self.frameElement).style("height", diameter + "px");
});